Insert the following packages before running the code:
1/ cv2
2/ tqdm
3/ densratio
I

In [1]:
''' Import relevant libraries'''
import cv2 # using opencv to process image
from tqdm import tqdm  # use tqdm to know the process of the iteration

import numpy as np
import pandas as pd
import os         
import random
import matplotlib.pyplot as plt

#from densratio import densratio

import time
from sklearn.metrics import accuracy_score
from sklearn.decomposition import PCA
from keras.models import Sequential
from keras.layers import Dense, Activation
from keras.utils import np_utils
import random

Using Theano backend.
WARNING (theano.configdefaults): g++ not available, if using conda: `conda install m2w64-toolchain`
WARNING (theano.configdefaults): g++ not detected ! Theano will be unable to execute optimized C-implementations (for both CPU and GPU) and will default to Python implementations. Performance will be severely degraded. To remove this warning, set Theano flags cxx to an empty string.


In [2]:
%matplotlib inline

train_path = r'C:\Users\phuph\Desktop\Spiderdatabase\spidertrainset'
csv_path = r'C:\Users\phuph\Desktop\Spiderdatabase\spider_csv.csv'
#test_path = 'test'

size_img = 50


In [3]:
""" From the image, record the label either dangerous or non"""
def record_label(image_name):
    label = image_name.split('.')[0]
    if label == 'dangerous':
        return 1
    elif label == 'non':
        return 0

def create_train():
    training_data = []
    for img in tqdm(os.listdir(train_path)):
        path = os.path.join(train_path, img)
        img_data = cv2.imread(path, cv2.IMREAD_COLOR).astype(float)
        img_data = cv2.resize(img_data, (size_img, size_img))
        training_data.append([img_data, record_label(img) ])
    #np.save('train_data.npy', training_data)
    return training_data


In [4]:
train = create_train()

100%|██████████████████████████████████████████████████████████████████████████████| 1005/1005 [00:16<00:00, 59.33it/s]


In [5]:
A_train = []
for i in train:
    a = i[0].reshape(-1)
    A_train.append(a)
A_train = np.asarray(A_train)
B_train = np.asarray([i[1] for i in train])

In [6]:
indices = np.arange(A_train.shape[0])
from sklearn.cross_validation import train_test_split
X_train0, X_test0, Y_train, Y_test,idx1,idx2 = train_test_split(A_train, B_train,indices, test_size=0.2, random_state=0)

C:\Users\phuph\Anaconda3\lib\site-packages\sklearn\cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [7]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
X_train1 = scaler.fit_transform(X_train0)
X_test1 = scaler.fit_transform(X_test0)

pca = PCA(n_components=20)
X_train = pca.fit_transform(X_train1)
X_test= pca.transform(X_test1)


In [8]:
''' Induce CCN flip rate'''

import random
rho_po_list = [0.1,0.2,0.3,0.4,0.4]
rho_ne_list = [0.3,0.1,0.4,0.1,0.4]
Y_new_list = []
for i in range(5):
    rho_po = rho_po_list[i]
    rho_ne = rho_ne_list[i]
    Y_temp = np.copy(Y_train)
    for j in range(len(Y_temp)):
        if Y_train[j] ==1:
            temp = random.random()
            if temp < rho_po:
                Y_temp[j] = 0
        else:
            t = random.random()
            if t < rho_ne:
                Y_temp[j] = 1
    Y_new_list.append([Y_temp,rho_po,rho_ne])
'''
Y_new_list = []
for i in range(5):
    rho_po = random.choice(rho_po_list)
    rho_ne = random.choice(rho_ne_list)
    Y_temp = Y_train * (np.cumsum(Y_train) <= (1 - rho_po) * sum(Y_train))
    s = 1 - (1 - Y_train) * (np.cumsum(1 - Y_train) <= (1 - rho_ne) * sum(1 - Y_train))
    Y_temp[Y_train==0] = s[Y_train==0]
    Y_new_list.append([Y_temp,rho_po,rho_ne])
'''

'\nY_new_list = []\nfor i in range(5):\n    rho_po = random.choice(rho_po_list)\n    rho_ne = random.choice(rho_ne_list)\n    Y_temp = Y_train * (np.cumsum(Y_train) <= (1 - rho_po) * sum(Y_train))\n    s = 1 - (1 - Y_train) * (np.cumsum(1 - Y_train) <= (1 - rho_ne) * sum(1 - Y_train))\n    Y_temp[Y_train==0] = s[Y_train==0]\n    Y_new_list.append([Y_temp,rho_po,rho_ne])\n'

In [9]:
''' Import R and KLIEP'''

import rpy2
import rpy2.robjects as robjects
r = robjects.r
#from rpy2.robjects.packages import importr
#utils = importr("densratio")
import rpy2.robjects.numpy2ri as numpy2ri
rpy2.robjects.numpy2ri.activate()

robjects.r('''
           f <- function(t,t1) {

                    library(densratio)
                    dens <- densratio(x = t, y = t1, method = "KLIEP")
                    result <-dens$compute_density_ratio(t1)
            }
            ''')
kliep_check = robjects.globalenv['f']

In [10]:
### Estimate the noise rate###
def KLIEP_e(X,Y):
    e_list=[]
    X_rho_po= X[Y ==1]
    X_rho_ne= X[Y ==0]
    result_po= kliep_check(X_rho_po,X)
    result_ne= kliep_check(X_rho_ne,X)
    Po = np.asarray(result_po)
    Ne = np.asarray(result_ne)
    n = len(X)
    py_po = sum(1 for i in Y if i==1)/n
    py_ne = sum(1 for i in Y if i==0)/n
    Po_n = min(Po*py_po)
    Ne_n = min(Ne*py_ne)
    e_list.append([Ne_n,Po_n])
    return e_list
    

In [11]:
''' Using KLIEP to estimate betas'''

def KLIEP_betas(X,Y):
    e_list=[]
    X_rho_po= X[Y ==1]
    X_rho_ne= X[Y ==0]
    result_po= kliep_check(X_rho_po,X)
    result_ne= kliep_check(X_rho_ne,X)
    Po = np.asarray(result_po)
    Ne = np.asarray(result_ne)
    n = len(X)
    py_po = sum(1 for i in Y if i==1)/n
    py_ne = sum(1 for i in Y if i==0)/n
    Po_n = min(Po*py_po)
    Ne_n = min(Ne*py_ne)
    betas = np.ones((n,), dtype = None)
    for i in range(n):
        if Y[i] == 0:
            if Po[i] == 0:
                betas[i] = 0
            else:
                betas[i] = (Po[i]*py_po - Po_n)/ ((1-Po_n-Ne_n)*Po[i]*py_po)
        else: 
            if Ne[i] == 0:
                betas[i] =0
            else:
                betas[i] = (Ne[i]*py_ne - Ne_n)/ ((1-Po_n-Ne_n)*Ne[i]*py_ne)

    return betas    

In [12]:
''' Neural networks model'''

dims = X_train.shape[1]
n_y = 2
print( 'Number of features: %d ' %dims)
print( 'Number of classes: %d' %n_y)
print("Building model...")

model = Sequential()
model.add(Dense(80, input_shape=(dims,)))
model.add(Dense(60, input_shape=(dims,)))
model.add(Dense(40, input_shape=(dims,)))
model.add(Dense(n_y, input_shape=(dims,)))
model.add(Activation('softmax'))
model.compile(optimizer='sgd', loss='categorical_crossentropy',sample_weight_mode=None,metrics=['accuracy'])


Number of features: 20 
Number of classes: 2
Building model...


In [21]:
# CCN - Cheng17 + random using LR as noise estimation##
from sklearn.linear_model import LogisticRegression
clf = LogisticRegression(penalty="l2")

accuracy_list =[]
for i in Y_new_list:
    r = [0.1, 0.2, 0.3, 0.4]
    rho_po_max = i[1]
    rho_ne_max = i[2]
    n = len(X_train)
    clf.fit(X_train,i[0])
    e = clf.predict_proba(X_train)
    eta_po = np.asarray([i[1] for i in e])
    St = []
    St_n=[]
    distill_ind = []

    for t in range(n):
        if eta_po[t] > ((1+rho_ne_max)/2):
            St.append([X_train[t],1])
            distill_ind.append([idx1[t],1])

        elif eta_po [t]  < ((1-rho_po_max)/2):
            St.append([X_train[t],0])
            distill_ind.append([idx1[t],0])
        else: 
            St_n.append([X_train[t],Y_train[t]])
    
    q = 0
    while (q<20):
        z = random.randint(0,len(St_n))
        St.append(St_n[z])
        q = q+1
    
      
    St_x =np.asarray([i[0] for i in St])
    St_y =np.asarray([i[1] for i in St])
    
    print(St_y)
    
    count =0
    sample_weight = KLIEP_betas(St_x,St_y)
    y= np_utils.to_categorical(St_y, 2)
    model.fit(St_x,y,sample_weight=sample_weight,verbose=1,epochs=1)
    pred = model.predict_classes(X_test)
    a = accuracy_score(Y_test,pred)
    accuracy_list.append([count,a,rho_po_max,rho_ne_max])
    count = count +1
    
for i in accuracy_list:
    print(i)


[1 0 0 0 0 1 1 1 1 1 1 0 0 1 1 0 1 1 0 1 1 0 1 0 1 1 1 1 1 0 1 1 1 1 1 0 1
 1 0 1 1 0 1 0 0 1 1 1 1 1 1 1 1 1 1 0 0 1 1 1 0 1 1 1 0 0 1 0 0 0 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 0 0 0 1 1 1 1 1 0 1 0 0 0 1 0 0 1 1 0 1 1 0 1 0
 1 0 1 1 0 1 1 0 1 1 1 1 1 0 1 1 1 1 1 0 0 1 1 1 1 0 0 1 1 1 1 1 1 1 1 0 0
 1 1 1 1 0 1 0 1 1 0 1 1 1 0 0 1 1 0 1 1 0 1 0 0 1 1 1 1 1 1 0 0 0 0 1 1 1
 1 0 0 0 1 1 1 0 1 1 1 1 0 0 0 1 1 0 1 0 1 0 1 0 0 1 1 0 1 1 0 1 0 1 0 1 1
 1 0 0 0 1 0 1 1 1 1 1 1 0 0 0 0 1 0 1 0 0 1 1 0 1 0 0 0 1 0 0 1 1 0 1 0 1
 0 1 1 0 1 1 1 1 1 1 0 1 1 1 0 1 0 0 1 1 1 1 1 1 1 1 1 1 0 1 1 1 1 0 0 1 1
 0 0 0 1 1 1 1 1 0 1 1 1 1 1 1 0 1 1 0 0 0 1 0 0 1 0 1 0 0 1 1 1 1 1 1 1 0
 1 1 1 1 1 0 0 0 0 0 1 1 1 1 0 1 1 1 1 1 1 1 0 0 1 1 1 0 1 1 1 1 1 1 1 1 0
 0 0 0 1 1 1 1 1 1 1 1 1 1 0 1 0 1 1 1 1 1 1 0 1 1 1 1 0 1 1 0 0 0 1 1 1 1
 1 1 1 0 0 1 1 1 1 0 1 1 1 0 1 0 1 1 1 0 1 1 1 0 1 0 1 0 1 0 0 1 1 1 1 1 1
 1 1 1 0 0 1 1 0 1 0 1 1 1 1 0 0 1 1 1 1 1 0 0 1 0 0 1 1 1 0 0 0 0 1 1 1 0
 0 1 0 0 1 0 1 1 1 1 1 0 

In [24]:
### CCN - Cheng17 auto + act   ###

from sklearn.linear_model import LogisticRegression
clf = LogisticRegression(penalty="l2")

accuracy_list =[]
for i in Y_new_list:
    r = [0.1, 0.2, 0.3, 0.4]
    rho_po_max = i[1]
    rho_ne_max = i[2]
    n = len(X_train)
     
    clf.fit(X_train,i[0])
    e = clf.predict_proba(X_train)
    eta_po = np.asarray([i[1] for i in e])
    d = clf.decision_function(X_train)

    St = []
    St_n=[]
    St_d=[]
    distill_ind = []

    for t in range(n):
        if eta_po[t] > ((1+rho_ne_max)/2):
            St.append([X_train[t],1])
            distill_ind.append([idx1[t],1])

        elif eta_po [t]  < ((1-rho_po_max)/2):
            St.append([X_train[t],0])
            distill_ind.append([idx1[t],0])
        else: 
            St_n.append([X_train[t],Y_train[t]])
            St_d.append(d[t])
    dist = np.abs(St_d)
    a = dist.argsort()[:30]
    for z in a:
        St.append(St_n[z])
    
    St_x =np.asarray([i[0] for i in St])
    St_y =np.asarray([i[1] for i in St])
    
    print(St_y)
    count =0
    sample_weight = KLIEP_betas(St_x,St_y)
    y= np_utils.to_categorical(St_y, 2)
    model.fit(St_x,y,sample_weight=sample_weight,verbose=1,epochs=1)
    pred = model.predict_classes(X_test)
    a = accuracy_score(Y_test,pred)
    accuracy_list.append([count,a,rho_po_max,rho_ne_max])
    count = count +1
    
for i in accuracy_list:
    print(i)


[1 0 0 0 0 1 1 1 1 1 1 0 0 1 1 0 1 1 0 1 1 0 1 0 1 1 1 1 1 0 1 1 1 1 1 0 1
 1 0 1 1 0 1 0 0 1 1 1 1 1 1 1 1 1 1 0 0 1 1 1 0 1 1 1 0 0 1 0 0 0 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 0 0 0 1 1 1 1 1 0 1 0 0 0 1 0 0 1 1 0 1 1 0 1 0
 1 0 1 1 0 1 1 0 1 1 1 1 1 0 1 1 1 1 1 0 0 1 1 1 1 0 0 1 1 1 1 1 1 1 1 0 0
 1 1 1 1 0 1 0 1 1 0 1 1 1 0 0 1 1 0 1 1 0 1 0 0 1 1 1 1 1 1 0 0 0 0 1 1 1
 1 0 0 0 1 1 1 0 1 1 1 1 0 0 0 1 1 0 1 0 1 0 1 0 0 1 1 0 1 1 0 1 0 1 0 1 1
 1 0 0 0 1 0 1 1 1 1 1 1 0 0 0 0 1 0 1 0 0 1 1 0 1 0 0 0 1 0 0 1 1 0 1 0 1
 0 1 1 0 1 1 1 1 1 1 0 1 1 1 0 1 0 0 1 1 1 1 1 1 1 1 1 1 0 1 1 1 1 0 0 1 1
 0 0 0 1 1 1 1 1 0 1 1 1 1 1 1 0 1 1 0 0 0 1 0 0 1 0 1 0 0 1 1 1 1 1 1 1 0
 1 1 1 1 1 0 0 0 0 0 1 1 1 1 0 1 1 1 1 1 1 1 0 0 1 1 1 0 1 1 1 1 1 1 1 1 0
 0 0 0 1 1 1 1 1 1 1 1 1 1 0 1 0 1 1 1 1 1 1 0 1 1 1 1 0 1 1 0 0 0 1 1 1 1
 1 1 1 0 0 1 1 1 1 0 1 1 1 0 1 0 1 1 1 0 1 1 1 0 1 0 1 0 1 0 0 1 1 1 1 1 1
 1 1 1 0 0 1 1 0 1 0 1 1 1 1 0 0 1 1 1 1 1 0 0 1 0 0 1 1 1 0 0 0 0 0 1 1 0
 0 0 0 1 0 0 0 0 0 0 0 1 

In [ ]:
## ILN - Cheng17 + rd using KLIEP"
from sklearn.model_selection import StratifiedKFold

accuracy_list =[]
df_noise = pd.read_csv(csv_path)
C_train = np.asarray(df_noise["Rater label"])

skf = StratifiedKFold(n_splits=10)
for train_index,test_index in skf.split(A_train,C_train,B_train):
    X_train0, X_test0 = A_train[train_index], A_train[test_index]
    Y_train, Y_test = C_train[train_index], C_train[test_index]
    BB_train, BB_test = B_train[train_index], B_train[test_index]
    idx1= train_index
    X_train1 = scaler.fit_transform(X_train0)
    X_test1 = scaler.fit_transform(X_test0)

    X_train = pca.fit_transform(X_train1)
    X_test= pca.transform(X_test1)

    r = [0.1,0.15, 0.2, 0.25, 0.3,0.35, 0.4, 0.45]
    #rho_po_max = 0.2
    #rho_ne_max = 0.45
    rho_po_max = random.choice(rho_po_list)
    rho_ne_max = random.choice(rho_ne_list)
    X_rho_po= X_train[Y_train ==1]
    result_po= kliep_check(X_rho_po,X_train)
    Po = np.asarray(result_po)
    n = len(X_train)
    py_po = sum(1 for z in Y_train if z==1)/n
    eta_po = Po*py_po

    St = []
    St_n=[]
    distill_ind = []

    for t in range(n):
        if eta_po[t] > ((1+rho_ne_max)/2):
            St.append([X_train[t],1])
            distill_ind.append([idx1[t],1])

        elif eta_po [t]  < ((1-rho_po_max)/2):
            St.append([X_train[t],0])
            distill_ind.append([idx1[t],0])
        else: 
            St_n.append([X_train[t],BB_train[t]])


    q = 0
    while (q<20):
        z = random.randint(0,len(St_n))
        St.append(St_n[z])
        q = q+1

    St_x =np.asarray([t[0] for t in St])
    St_y =np.asarray([t[1] for t in St])
    print(St_y)
    sample_weight = KLIEP_betas(St_x,St_y)
    y= np_utils.to_categorical(St_y, 2)
    model.fit(St_x,y,sample_weight=sample_weight,verbose=1,epochs=1)
    pred = model.predict_classes(X_test)
    a = accuracy_score(Y_test,pred)
    accuracy_list.append([a])

print(np.mean(accuracy_list,axis=0),np.std(accuracy_list,axis=0))

In [56]:
### ILN - Cheng17 + random using LR  ###
from sklearn.model_selection import StratifiedKFold

accuracy_list =[]
df_noise = pd.read_csv(csv_path)
C_train = np.asarray(df_noise["Rater label"])

skf = StratifiedKFold(n_splits=10)
for train_index,test_index in skf.split(A_train,C_train,B_train):
    x_train0, x_test0 = A_train[train_index], A_train[test_index]
    y_train, y_test = C_train[train_index], C_train[test_index]
    BB_train, BB_test = B_train[train_index], B_train[test_index]
    idx1= train_index
    x_train1 = scaler.fit_transform(x_train0)
    x_test1 = scaler.fit_transform(x_test0)

    x_train = pca.fit_transform(x_train1)
    x_test= pca.transform(x_test1)

    r = [0.1,0.15, 0.2, 0.25, 0.3,0.35, 0.4, 0.45]
    rho_po_max = random.choice(rho_po_list)
    rho_ne_max = random.choice(rho_ne_list)
    n = len(x_train)
    clf.fit(x_train,y_train)
    e = clf.predict_proba(x_train)
    eta_po = np.asarray([i[1] for i in e])
    
    St = []
    St_n=[]
    distill_ind = []

    for t in range(n):
        if eta_po[t] > ((1+rho_ne_max)/2):
            St.append([x_train[t],1])
            distill_ind.append([idx1[t],1])

        elif eta_po [t]  < ((1-rho_po_max)/2):
            St.append([x_train[t],0])
            distill_ind.append([idx1[t],0])
        else: 
            St_n.append([x_train[t],BB_train[t]])

    
    q = 0
    while (q<30):
        z = random.randint(0,len(St_n))
        St.append(St_n[z])
        q = q+1
    
    St_x =np.asarray([t[0] for t in St])
    St_y =np.asarray([t[1] for t in St])
    #print(St_y)
    sample_weight = KLIEP_betas(St_x,St_y)
    y= np_utils.to_categorical(St_y, 2)
    model.fit(St_x,y,sample_weight=sample_weight,verbose=1,epochs=1)
    pred = model.predict_classes(x_test)
    a = accuracy_score(y_test,pred)
    accuracy_list.append([a])

print(np.mean(accuracy_list,axis=0),np.std(accuracy_list,axis=0))

Epoch 1/1
 32/100 [========>.....................] - ETA: 0s[ 0.53725743] [ 0.10461637]


In [49]:
### ILN - Cheng17 auto + act using LR  ###

from sklearn.linear_model import LogisticRegression
clf = LogisticRegression(penalty="l2")

from sklearn.model_selection import StratifiedKFold

accuracy_list =[]
df_noise = pd.read_csv(csv_path)
C_train = np.asarray(df_noise["Rater label"])

skf = StratifiedKFold(n_splits=10)
for train_index,test_index in skf.split(A_train,C_train, B_train):
    x_train0, x_test0 = A_train[train_index], A_train[test_index]
    y_train, y_test = C_train[train_index], C_train[test_index]
    BB_train, BB_test = B_train[train_index], B_train[test_index]
    idx1= train_index
    x_train1 = scaler.fit_transform(x_train0)
    x_test1 = scaler.fit_transform(x_test0)

    x_train = pca.fit_transform(x_train1)
    x_test= pca.transform(x_test1)

    r = [0.1,0.15, 0.2, 0.25, 0.3,0.35, 0.4, 0.45]
    rho_po_max = random.choice(rho_po_list)
    rho_ne_max = random.choice(rho_ne_list)
    n = len(x_train)
    clf.fit(x_train,y_train)
    e = clf.predict_proba(x_train)
    eta_po = np.asarray([i[1] for i in e])
    d = clf.decision_function(x_train)
        
    
    St = []
    St_n=[]
    distill_ind = []
    St_d=[]

    for t in range(n):
        if eta_po[t] > ((1+rho_ne_max)/2):
            St.append([x_train[t],1])
            distill_ind.append([idx1[t],1])

        elif eta_po [t]  < ((1-rho_po_max)/2):
            St.append([x_train[t],0])
            distill_ind.append([idx1[t],0])
        else: 
            St_n.append([x_train[t],BB_train[t]])
            St_d.append(d[t])
     
    dist = np.abs(St_d)
    a = dist.argsort()[:20]
    for z in a:
        St.append(St_n[z])
  

    St_x =np.asarray([t[0] for t in St])
    St_y =np.asarray([t[1] for t in St])
    #print(St_y)
    sample_weight = KLIEP_betas(St_x,St_y)
    y= np_utils.to_categorical(St_y, 2)
    model.fit(St_x,y,sample_weight=sample_weight,verbose=1,epochs=1)
    pred = model.predict_classes(x_test)
    a = accuracy_score(y_test,pred)
    accuracy_list.append([a])

print(np.mean(accuracy_list,axis=0),np.std(accuracy_list,axis=0))

Epoch 1/1
 32/100 [========>.....................] - ETA: 0s[ 0.53130693] [ 0.08355827]
